# TP donnéees multimédia - CES2016
Stéphane DONNE    
Guillaume DANJOU    
Arthur SECK   
Mame-Mor FALL    

**Objectifs du TP :**
O1 définir ce que sont les données multimedia (terme multimedia, qui a plusieurs assertions + exemples
variés, enjeux liés aux masses de données)
O2 extraire des caractéristiques propres aux données sonores (pitch, MFCC, etc. ) et visuelles (couleur,
forme, texture, notamment SIFT et "sacs de mots" et caractéristiques propres aux visages). Attention il
faudra sans doute revoir quelques bases (Transformée de Fourier, notion de fréquence)
O3 mesurer l'intérêt de ces caractéristiques ; savoir poser le problème plus que donner toutes les
réponses : suggestions de quelques heuristiques et ouverture vers la sélection automatique ; aspects
subjectifs (notion de fossé sémantique)
O4 réutiliser de façon autonome ces savoirs dans un projet personnel

● synthèse des connaissances acquises dans ce module (détaillées) ;
● discussion critique de leur mise en œuvre sur l’activité vidéo (démonstration d’une réflexion
personnelle face à des problèmes concrets) ;
● description de la mise en œuvre dans le projet personnel (éventuellement) ;
● 1 question que l’on aurait aimé approfondir (pour prendre du recul).

**O1 définir ce que sont les données multimedia (terme multimedia, qui a plusieurs assertions + exemples variés, enjeux liés aux masses de données) :**

Les "données multimédia" au sens large sont l'ensemble des informations que l'on peut extraire de contenus audiovisuels. Les exemples de contenus audiovisuels et de cas d'analyses n'ont de limite que l'imagination humaine  :
- Transcrire un texte à partir de signaux de paroles humaine (speech to text translation, ou traduire en temps réel les paroles dans une autre langue speech to speech translation comme proposé par Skype)
- Retrouver le titre et les auteurs à partir d'une bande sonore musicale (Shazam)
- Identifier des visages dans une image (appareils photo, Facebook propose d'identifier automatiquement les visages de nos amis sur les photos, Google Photos propose de classer les albums par personne identifié etc.)
- Identifier des actions dans une vidéo (travaux de recherche en cours ex:http://lear.inrialpes.fr/)
- etc.

Aujourd'hui ces contenus dont la disponibilité explose sur Internet sont l'objet d'enjeux collosaux pour leur stockage et leur accès rapide premièrement, et pour leur analyse deuxièmement afin de proposer de la valeur commerciale par l'automatisation de tâche ou la recommandation de contenus.  

**O2 extraire des caractéristiques propres aux données sonores (pitch, MFCC, etc. ) et visuelles (couleur, forme, texture, notamment SIFT et "sacs de mots" et caractéristiques propres aux visages). Attention il faudra sans doute revoir quelques bases (Transformée de Fourier, notion de fréquence) :**

L'objectif que l'on se donne dans cette séquence est d'arriver à retrouver plusieurs séquences à l'intérieur d'un enregistrement de l'émission vidéo "le débat".
- les transitions musique/parole
- les différents locuteurs au sein de l'émission

Extraction de la bande sonore :
La bande sonore peut être extraite par l'intermédiaire de plusieurs logiciels libres disponibles (ex: ffmpeg). Pour ce TP elle nous est déjà donnée en séance en format WAVE (Waveform Audio File Format) stéréo échantillonné à 44100 bit/secondes.

Nous pouvons donc directement commencer à analyser la bande sonore au travers de python.

La stratégie consiste à exploiter les coefficients cespraux tirés du signal sonore et à les utiliser pour construire un modèle de classification. La base d'apprentissage sera composée des premières minutes d'enregistrement.

Le signal sonore est stocké dans un numpy array. On utilise la bibliothèque librosa. Pour que le programme tourne rapidement, on se limite à 10 minutes d'enregistrement avec le paramètre "duration", mais on pourrait faire tourner le programme sur la durée entière.

In [1]:
"""
Created on Sun Apr  3 17:18:11 2016

@author: Stéphane, Guillaume, Arthur, Mame-Mor
"""
import numpy as np
import scipy
import matplotlib.pyplot as plt
import librosa
from scipy.io import wavfile

%matplotlib inline
duration=180
y, sr = librosa.core.load("/home/reddowan/Documents/Telecom Paris/4-Multimedia/06-11-22.wav", sr=44100, duration=duration)
print "la frequence d'echantillonnage est", sr
print "la taille de l'array y est de", y.shape
librosa.display.waveplot(y,sr)
plt.title('Amplitude de la bande sonore')
plt.xlabel('temps')
plt.ylabel('amplitude')
plt.show()
"""On observe la representation des variations d'amplitude de la bande sonore en fonction du temps"""

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-1-ee49030227a3>, line 15)

In [ ]:
z = scipy.fft(y)
z_mag=np.absolute(z)
plt.figure()
plt.plot(z_mag)
plt.title('Amplitudes spectrales')
plt.xlabel('frequences')
plt.ylabel('amplitude')
plt.show()

Après ces deux étapes de visualisation du signal nous allons construire une matrice de caractéristiques du signal. Cette matrice contiendra en colonne les coefficients cespraux calculés à l'aide de l'algorithme MFCC (representation filtre-source de la bande spectrale basée sur la psychoacoustique qui permet une bonne discrimination des signaux de paroles) et le Zero Cross Rate (taux de passage par zéro du signal qui est une dimension discriminante pour la musique et la voix). 

On normalisera ces différents paramètres avant d'essayer plusieurs algorithmes pour classifier les différentes séquences.
On essaiera un algorithme de classification non supervisé, le clustering par K-means.
Puis on entraînera un classifieur supervisé le LDA.
Enfin nous comparerons les résultats et élaborerons sur ce que nous aurions aimé faire pour aller plus loin.

## Création de la matrice de features
### Calcul du ZCR
Avec le ZCR on espère obtenir un paramètre efficace pour séparer les séquences de paroles des séquences de musiques. Le ZCR se calcule sur une fenêtre glissante qui a une taille (frame_length) et glisse de n échantillons entre deux itérations (hop_length). Les mêmes paramètres seront utilisés pour le calcul des coefficients cepstraux avec le MFCC. Le choix de ces paramètres est important car on comprend intuitivement qu'une fenêtre trop grande moyenne les évolutions du signal, on pourrait ainsi avoir une fenêtre qui comprend toujours plusieurs séquences à la fois et dans ce cas impossible d'avoir une discrimination efficace. Dans le sens inverse une fenêtre trop petite  entraînera du bruit car nous seront ammènés à voir des changements de séquence à un niveau de détail qui ne nous intéresse pas. Idéalement nous voulons isoler 6 séquences, 5 locuteurs (1 présentatrice + 4 intervenants) et la musique de l'emission.
Pour trouver les paramètres de fenêtre optimaux nous pourrions en tester plusieurs et trouver ceux qui isole le mieux ces 6 séquences mais cela serait très consommateur de ressources. Nous essaierons donc arbitrairement une fenêtre d'1 seconde et un hop length de 0.5 seconde qui nous semble les performances qu'un humain aurait a discriminer les différentes séquences.

In [ ]:
n_fft=44100
hop_length=22050
sr=44100

In [ ]:
zero_crossing_rate = librosa.feature.zero_crossing_rate(y, frame_length=n_fft, hop_length=hop_length)
plt.plot(zero_crossing_rate[0])
plt.title('Zero Crossing Rate')
plt.xlabel('Echantillons (par demi-seconde)')
plt.ylabel('Taux')
zero_crossing_rate.shape

SUr le graphique ci-dessus on observe bien une différence de pattern entre les 10 premières secondes de musique et le reste du signal.

### Calcul du MFCC
Avec les coefficients MFCC on souhaite séparer les différents locuteurs au sein des signaux de parole. Comme le ZCR, le melspectrogram se calcule sur une fenêtre glissante dont nous conserverons les paramètres utilisés précédemment.

In [ ]:
S=librosa.feature.melspectrogram(y=y,sr=sr,n_fft=n_fft,hop_length=hop_length)
print S.shape
logS=librosa.logamplitude(S)
librosa.display.specshow(logS[0:13,:],sr=sr,y_axis='mel')
plt.title('Representation du melspectrogram')
plt.show()
"""Zoom sur les 13 premiers coefficients cepstraux"""

On agrège les paramètres calculés en une matrice des paramètres.

In [ ]:
Features=np.concatenate((zero_crossing_rate,logS[0:13,:]),axis=0)
print (Features.shape)

### Normalisation des données
On normalise les 14 paramètres de notre matrice pour améliorer les résultats de notre future classification et éviter le surapprentissage sur quelques paramètres.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Features_norm = scaler.fit_transform(Features)

### Clustering des séquences
Dans cette section nous allons essayer de produire la segmentation de l'extrait en différente séquence de manière non-supervisée. C'est à dire que nous n'allons pas indiquer à l'algorithme les labels recherchés et leurs caractéristiques par de l'apprentissage. L'algorithme de clustering va séparer l'extrait sonore en différentes parties les plus "proches" au sens de la distance dans l'espace de représentation. Plusieurs algorithme de clustering existe, celui proposé par librosa s'appuie sur la librairie scikit learn et s'appelle agglomerative clustering. Dans le cas de 2 classes, cet algorithme parcourt l'extrait sonore pour trouver la séparation temporelle qui minime la distance de chacune des observations de caractéristiques au centre de classe ainsi déterminé. Idem avec plusieurs classes. L'idée clé est que les observations de classes sont continues.

In [ ]:
boundary_frames=librosa.segment.agglomerative(Features_norm,2)
librosa.frames_to_time(boundary_frames,sr=sr)
print(boundary_frames*500)
librosa.display.waveplot(y,sr,x_axis='time')
plt.vlines(boundary_frames*500, ymin=-1,ymax=1, color='r', linestyles='solid', label='Segment boundaries')
plt.legend(frameon=True, shadow=True)
plt.title('Waveplot et cluster')
plt.tight_layout()

Ce premier résultat est très intéressant. On constate en vérifiant sur l'enregistrement sonore que les deux séquences séparées par l'algorithme correspondent bien à la disparition complète de la musique en fond sonore pour marque le début de l'émission où la présentatrice prend la parole. En réalité un auditeur humain aurait marqué la fin de la musique aux alentours des 10 secondes quand l'introduction commence, mais durant tout le reportage la musique continue en arrière plan. Avec notre normalisation de la matrice, l'intensité des coefficients n'entre plus en compte et ce résultat semble tout à fait cohérent.

On ajoute un cluster supplémentaire et on observe le résultat :

In [ ]:
boundary_frames=librosa.segment.agglomerative(Features_norm,3)
librosa.frames_to_time(boundary_frames,sr=sr)
print(boundary_frames*500)
librosa.display.waveplot(y,sr,x_axis='time')
plt.vlines(boundary_frames*500, ymin=-1,ymax=1, color='r', linestyles='solid', label='Segment boundaries')
plt.legend(frameon=True, shadow=True)
plt.title('Waveplot et cluster')
plt.tight_layout()

Nous obtenons à nouveau un résultat parfaitement interprétable, la nouvelle séparation identifiée par l'algorithme correspond à la fin du jingle de début d'émission et au démarrage du reportage d'introduction. Ceci valide la pertinence d'une telle approche pour identifier différentes séquences dans un extrait sonore, de plus l'avantage d'une méthode non-supervisée est sa généralisation possible à d'autres émissions par exemple, si tenté que nous ayons l'hyperparamètre k du nombre de séquences recherchées.

### Apprentissage supervisé par Linear Discriminant analysis

Pour réaliser un apprentissage supervisé nous devons créer des étiquettes cibles que nous tacherons de retrouver sur des nouvelles données qui n'ont pas été utilisée pour l'apprentissage. Pour cette partie l'objectif va être de retrouver le genre du locuteur (Homme ou Femme), on crée donc un numpy array, nommé genre, contenant le genre du locuteur à chaque instant. Pour retrouver facilement dans ce numpy array genre un instant de l'enregistrement sous forme de minute et seconde, deux paramètres minute et seconde sont alimentés.

Les caractéristiques que nous utiliserons pour cette partie seront les coefficients cesptraux obtenus par MFCC.

In [ ]:
z, sr = librosa.core.load(sr = 44100, path='D:/Formations/Telecom_ParisTech/Module_04/06-11-22.wav', duration=10*60)
Z = scipy.fft(z)
Z_mag=np.absolute(Z)
S=librosa.feature.melspectrogram(z,sr=sr)
logS=librosa.logamplitude(S)
genre=np.zeros(len(logS.T))
X=np.zeros(len(logS.T))

In [ ]:
print(genre.shape)

In [ ]:
def coef_cresp_minute_seconde(coef_min,coef_max,minute_debut, seconde_debut,minute_fin,seconde_fin):
    librosa.display.specshow(logS[coef_min:coef_max,minute*minute_debut+seconde*seconde_debut:minute*minute_fin+seconde*seconde_fin],sr=sr,x_axis='time',y_axis='mel')
    plt.title('Representation du melspectrogram')
    plt.show()
    print(coef_min)
    print(coef_max)
    print(minute*minute_debut+seconde*seconde_debut)
    print(minute*minute_fin+seconde*seconde_fin)


Le melspectrogramme est tracé entre les temps 1mn00s et 4 mn 58s de l'extrait sonore. On semble percevoir la différence entre la séquence de 1 à 2 mn 59 s pendant laquelle la femme parle et la minute suivante où l'homme parle. Une autre femme parle entre 4 mn et 4 mn 58 s, le melspetgrogram est "proche" de celui de la première femme qui a parlé.

Cela laisse penser que les coefficients cesptraux permettront peut-être de différencier les hommes et les femmes.

In [ ]:
coef_cresp_minute_seconde(0,13,0,0,4,59)

On fournit en apprentissage les données de genre des minutes 1 à 5 de l'enregistrement. On tire parti du fait que, pendant ces moments, on entend distinctement une personne à la fois pendant un temps assez long. Le défaut pourrait être que le modèle "n'apprendrait pas bien" les séquences pendant lesquelles plusieurs personnes parlent en même temps, ainsi que les prises de parole très brèves.
On annote cette matrice de genre manuellement pour l'apprentissage.

In [ ]:
genre[minute*1+seconde*0:minute*3+seconde*0]=2         # une femme parle entre 1 mn 00s et 3 mn 00s (genre=2)
genre[minute*3+seconde*0:minute*3+seconde*52]=1        # un homme parle entre 3 mn 00s et 3 mn 52s  (genre=1)
genre[minute*3+seconde*52:minute*4+seconde*44]=2
genre[minute*4+seconde*44:minute*4+seconde*59]=1

Le genre du locuteur est tracé pendant les 5 premières minutes. La première minute (valeur 0) ne sera pas prise en compte, car elle contient de la musique.

In [ ]:
plt.ylim(0,3)
plt.plot(genre)

In [ ]:
print(logS.shape)

On crée les données pour l'apprentissage. Le numpy array X contient les n premiers coefficients cesptraux pendant la période 1mn00s à 4mn59s. y contient le genre du locuteur pendant cette période (de manière à travailler avec les notations habituelles X pour les features et y pour l'étiquette de classification)

Le numpy array "X_reste" contient le reste de l'enregistrement (de 5mn00s à 10mn00) pour tester le bon fonctionneemnt du modèle au-delà de la base de test (X_test, voir plus loin)

In [ ]:
nb_coef_cespraux = 13   #paramètres permettant de sélectionner les n premiers coefficients cespraux

X=logS.T[minute*1+seconde*0:minute*4+seconde*59,0:nb_coef_cespraux-1]
y = genre[minute*1+seconde*0:minute*4+seconde*59]
print(X.shape, y.shape, np.unique(y))

X_reste=logS.T[minute*5+seconde*0:51680,0:nb_coef_cespraux-1]
print(X_reste.shape)

plt.figure()
plt.plot(y)
plt.ylim(0,3)
plt.show()


Les données X sont standardisées pour améliorer les résultats du classifieur et éviter le surapprentissage sur une partie des coefficients de plus forte amplitude.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Xnorm = scaler.fit_transform(X)

In [ ]:
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score

On effectue une partition de X et y entre la partie des données que nous allons utiliser pour l'apprentissage (75 % de l'ensemble nommés X_train et y_train) et la partie que nous gardons pour le test (25 % de l'ensemble nommés X_test et y_test). Cette partition peut présenter un inconvénient si elle conduisait à retenir un trop petit nombre d'observation pour les hommes ou les femmes.
On pourrait pallier ce problème en s'assurant d'avoir des échantillons représentatif des deux genres en utilisant y. Nous ne le faisons pas ici, mais cela deviendrait très important si on essayait d'identifier le locuteur par exemple et non plus seulement son genre. 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Xnorm,y,train_size=0.75,random_state=20)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
LDA = LinearDiscriminantAnalysis()
LDA.fit(X_train, y_train)

y_pred = LDA.predict(X_test)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
print(np.mean(cross_val_score(LDA,X_train,y_train,cv=5)))
print(confusion_matrix(y_test, y_pred))

On obtient un taux de prédiction de 89 %.

On effectue la prévision sur la partie restante de l'enregistrement (5mn 00 à 9 mn 59s) pour aller au-delà de la base de test. Le but est de simuler ce qui se passerait si le classifieur était mis "en production". Les features de ces 5 minutes d'enregistrement supplémentaires sont stockées dans le numpy array X_reste (X_reste_norm pour les données standardisées). Y_reste contient la valeur du genre pour les minutes 5mn00s à 9mn59s d'enregistrement.

In [ ]:
X_reste_norm = scaler.transform(X_reste)
print(X_reste_norm.shape)

In [ ]:
X_reste_norm = scaler.transform(X_reste) # on applique la strandardisation utilisée sur l'échantillon train
y_reste = LDA.predict(X_reste_norm)           # prédiction du genre

print(X_reste_norm.shape)
print(y_reste.shape)

plt.figure()
plt.ylim(0,3)
plt.plot(y_reste)
plt.show()

Un problème apparaît immédiatement au vu du graphique ci-dessus. Les oscillations entre les valeurs prédites 1 et 2 sont grandes, y compris dans une même seconde (voir graphique ci-dessous avec la prédiction de la première seconde). Nous pensons que cela tient au caractère très fluctuant des caractéristiques temporelles du signal. Par voie de conséquence, les coefficients cespraux doivent être volatiles au cours du temps, d'où l'instabilité dans les résultats prédits.

In [ ]:
plt.figure()
plt.plot(y_reste[86*0:86*5])
plt.show()

Une idée, provenant de l'analyse seconde par seconde des graphiques, est de voir qu'il y a bien des oscillations, mais que, "la majorité du temps", une valeur est prise pour le genre plutôt que l'autre. Nous tentons donc la règle suivante : si, au cours d'une seconde de temps donnée, le clasiffieur prédit 1 plus de la moitié du temps, on affecte 1 sur la seconde entière, 2 dans le cas contraire. Cela permettra de lisser les valeurs obtenues. Ce calcul est stocké dans un numpy array nommé genre_reste.

Une autre approche serait d'élargir la fenètre glissante du calcul des coefficients cepstraux.

In [ ]:
genre_reste = np.zeros(y_reste.shape[0])
moyenne = np.zeros(y_reste.shape[0])

for i in range(0,y_reste.shape[0],seconde):
    moyenne[i]= np.mean(y_reste[i:i+seconde-1])
    if moyenne[i] < 1.5:
         genre_reste[i/seconde]=1
    else:
        genre_reste[i/seconde]=2     
    #print(i,i+seconde-1, round(np.mean(y_reste[i:i+seconde-1]),2), genre_reste[i])    

Effectivement, la figure ci-dessous (qui retrace les 5 minutes d'enregistrement entre 5mn00 et 9mn59s de l'émission) montre que le lissage a opéré. On voit en particulier une durée assez longue entre les secondes 70 et 150 (donc entre les temps 6mn10 et 7 mn 30s) pendant laquelle une femme parle (Juliette MATHYS-SIERRO, voir la vidéo).

In [ ]:
plt.figure()
plt.plot(genre_reste[0:300])
plt.show()

Avec l'écoute de la bande son, on construit le tableau des "vraies" valeurs pour le genre (dans un numpy array nommé true_genre_reste). Cela permettra une comparaison graphique avec la prévision (numpy array genre_reste, voir plus haut) et le calcul du pourcentage de temps sur lequel le genre du locuteur est correctement prédit.

Le tableau ci-dessous correspond à une annotation effectuée manuellement sur la base de l'écoute de l'enregistrement.

Minute	Seconde	Personne	Genre
5	0	Jérôme MORISOD	H
5	7	Isabelle GAY	F
5	9	Jérôme MORISOD	H
5	15	Isabelle GAY	F
5	19	Jérôme MORISOD	H
5	37	Isabelle GAY	F
5	42	Jérôme MORISOD	H
6	6	Isabelle GAY	F
6	17	Juliette MATHYS-SIERRO	F
6	42	Isabelle GAY	F
6	56	Juliette MATHYS-SIERRO	F
7	22	Isabelle GAY	F
7	23	Juliette MATHYS-SIERRO	F
7	32	Isabelle GAY	F
7	35	Damien KOENIG	H
8	17	Isabelle GAY	F
8	39	Damien KOENIG	H
8	53	Isabelle GAY	F
8	55	Damien KOENIG	H
9	7	Isabelle GAY	F
9	8	Damien KOENIG	H
9	11	Isabelle GAY	F
9	14	Damien KOENIG	H
9	46	Isabelle GAY	F
9	48	Brigitte BERTHOUZOZ	F
9	58	Isabelle GAY	F
10	0	Brigitte BERTHOUZOZ	F


In [ ]:
true_genre_reste = np.zeros(y_reste.shape[0])
true_genre_reste[0:7] = 1
true_genre_reste[7:9] = 2
true_genre_reste[9:15] = 1
true_genre_reste[15:19] = 2
true_genre_reste[19:37] = 1
true_genre_reste[37:42] = 2
true_genre_reste[42:66] = 1
true_genre_reste[66:77] = 2
true_genre_reste[77:102] = 2
true_genre_reste[102:116] = 2
true_genre_reste[116:142] = 2
true_genre_reste[142:143] = 2
true_genre_reste[143:152] = 2
true_genre_reste[152:155] = 2
true_genre_reste[155:197] = 1
true_genre_reste[197:219] = 2
true_genre_reste[219:233] = 1
true_genre_reste[233:235] = 2
true_genre_reste[235:247] = 1
true_genre_reste[247:248] = 2
true_genre_reste[248:251] = 1
true_genre_reste[251:254] = 2
true_genre_reste[254:286] = 1
true_genre_reste[286:288] = 2
true_genre_reste[288:298] = 2
true_genre_reste[298:300] = 2

On calcule le pourcentage de secondes dans la séquence de 5 mn (allant de 5mn00 à 9mn59s) pour lesquelles le genre du locuteur est bien prévu. On obtient 93,3 %, ce qui semble relativement correct. Il convient toutefois d'analyser les échecs pour voir si cela correspond à un cas particulier.

In [ ]:
print(100*np.sum(true_genre_reste[0:300]==genre_reste[0:300])/300)

L'examen des instants de mauvaise prédiction montre qu'il s'agit souvent du moment de "raccord" entre une voix féminine et une voix masculine. Il serait donc possible d'améliorer le taux de bonne prévision en annotant plus finement le genre du locuteur avec un temps mesuré en demi-secondes ou en dixièmes de secondes. 

In [ ]:
indices = np.where(true_genre_reste[0:300]!=genre_reste[0:300])
print(indices)

Plusieurs pistes d'amélioration de ce travail sont possibles :
1. Améliorer la qualité de prédiction par un réglage très fin des hyperparamètres par GridSearchCV
2. Essayer de réduire la durée de la plage d'apprentissage tout en obtenant un taux de prédiction correcte supérieur à 90 %.
3. Faire le lien entre ce module et le logiciel ELAN pour annoter automatiquement la vidéo.
4. Etendre ce travail à la prédiction de l'identité du locuteur et non plus seulement du sexe.
5. Déterminer si un autre type de classifieur (type KNN) n'aboutirait pas à de meilleures performances, voir ci-après.

### Apprentissage supervisé par KNN

In [ ]:
#KNN en choisissant k par validation croisée (V-fold) avec V = 6)
from sklearn import neighbors
from sklearn.cross_validation import cross_val_score

max_k = [1, 2, 3, 4, 5, 10, 20, 30, 40, 50]
scores = np.empty(len(max_k))

for k, d in enumerate(max_k):
    clf_KNN_optim = neighbors.KNeighborsClassifier(n_neighbors=d)
    scores[k] = np.mean(cross_val_score(clf_KNN_optim, X_train, y_train, cv=6, scoring='accuracy'))
best_score = scores.argmax()
best_k = max_k[best_score]
print "best k : " + str(best_k)

##Best KNN

clfKNN = neighbors.KNeighborsClassifier(n_neighbors=best_k)
clfKNN.fit(X_train, y_train)

y_predKNN = clfKNN.predict(X_test)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
print(np.mean(cross_val_score(clfKNN,X_train,y_train,cv=5)))
print(confusion_matrix(y_test, y_predKNN))

y_resteKNN = clfKNN.predict(X_reste_norm)           # prédiction du genre

print(X_reste_norm.shape)
print(y_resteKNN.shape)

plt.figure()
plt.ylim(0,3)
plt.plot(y_resteKNN)
plt.show()

Une fonction getGenreReste est créée. Elle a pour but de déterminer chaque seconde le genre du locuteur à partir de la moyenne prédite par le classifieur calculée sur une seconde.

In [ ]:
def getGenreReste(x, rate):
    output = np.zeros(x.shape[0])
    moyenne = np.zeros(x.shape[0])

    for i in range(0,x.shape[0],rate):
        moyenne[i]= np.mean(x[i:i+rate-1])
        if moyenne[i] < 1.5:
             output[i/rate]=1
        else:
            output[i/rate]=2
    return output

In [ ]:
print scores
genre_resteKNN = getGenreReste(y_resteKNN, seconde)

print(100*np.sum(true_genre_reste[0:300]==genre_resteKNN[0:300])/300)

Le choix définitif entre un classifieur de type KNN et LDA pourrait constituer un travail d'approfondissement de ce projet. Le KNN peut offrir des performances supérieures, mais il faut prendre garde à éviter le surapprentissage. Il serait intéressant de les comparer sur l'ensemble de l'enregistrement afin de trancher.

### Conclusion

Dans ce TP nous avons pu créer et manipuler plusieurs caractéristiques issue d'un enregistrement sonore. La représentation temporelle d'un signal d'amplitude échantillonné, sa représentation spectrale grâce à la transformée de Fourrier, puis d'autres caractéristiques en vue d'une classification de séquences dans l'enregistrement comme le ZCR par une fenêtre glissante et les coefficients cepstraux à travers la représentation MFCC. D'un enregistrement sonore nous pouvons donc utiliser de nombreuses représentations en fonction de l'étude souhaitée in fine.

Pour la classification des séquences nous avons utilisé une approche non supervisée par clustering et deux approches supervisée via un LDA et un KNN. Les trois méthodes ont donné des résultats satisfaisants et nous avons pu voir les avantages et limites de chacunes de ces méthodes.